In [19]:
import numpy as np

In [20]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=2#检测零件1的成本
    C2=3#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=6#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [21]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为109968036.2604253
第一阶段的生产决策为:
[0, 0, 0, 1, 186065.47000000003, 4685.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 87171.672695, 2195.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 0, 0, 56941.94, 0, 0.0, 0.0]
2
最终的最大利润为110492493.73019537
第一阶段的生产决策为:
[0, 0, 0, 1, 183867.535, 4718.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 86748.703013, 2226.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 0, 0, 57221.72999999999, 0, 0.0, 0.0]
3
最终的最大利润为110914397.00439975
第一阶段的生产决策为:
[0, 0, 0, 1, 181669.59999999998, 4751.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 86311.22696, 2257.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 0, 0, 57447.80000000001, 0, 0.0, 0.0]
4
最终的最大利润为111306555.18304016
第一阶段的生产决策为:
[0, 0, 0, 1, 179471.66500000004, 4784.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 85859.24453600001, 2288.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 0, 0, 57658.369999999995, 0, 0.0, 0.0]
5
最终的最大利润为111668702.35552031
第一阶段的生产决策为:
[0, 0, 0, 1, 177273.73000000004, 4816.0, 0.19, 0.19]
第二阶段的生产决策为:
[0, 0, 0, 1, 85375.028368, 2319.0, 0.19, 0.19]
第三阶段的生产决策为:
[1, 1, 0

In [22]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:96
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:29
在0.95-1.05波动时,决策差异率为:0.232


In [27]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1
    

1
最终的最大利润为104386830.39224018
第一阶段的生产决策为:
[0, 0, 0, 1, 199416.55999999994, 4486.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 89458.26881599998, 2012.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 53987.15999999999, 0, 0.0, 0.0]
2
最终的最大利润为105483395.33266014
第一阶段的生产决策为:
[0, 0, 0, 1, 194911.47999999992, 4553.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 88743.19684399998, 2073.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 54570.78, 0, 0.0, 0.0]
3
最终的最大利润为106460944.75200018
第一阶段的生产决策为:
[0, 0, 0, 1, 190406.40000000002, 4620.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87967.7568, 2134.0, 0.18000000000000002, 0.18000000000000002]
第三阶段的生产决策为:
[1, 1, 0, 0, 55092.40000000001, 0, 0.0, 0.0]
4
最终的最大利润为107441601.30224036
第一阶段的生产决策为:
[0, 0, 0, 1, 185901.31999999995, 4688.0, 0.18000000000000002, 0.18000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 87150.53881599996, 2197.0, 0.1800000000000

In [28]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:51
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:74
在0.9-1.1波动时,决策差异率为:0.592


In [29]:
n0=10000
p1=0.2
p2=0.2
p_product=0.2
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,0,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,0,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为92104400.62144014
第一阶段的生产决策为:
[0, 0, 0, 1, 227111.68, 4072.0, 0.16000000000000003, 0.16000000000000003]
第二阶段的生产决策为:
[0, 0, 0, 1, 92479.87609599998, 1658.0, 0.16000000000000003, 0.16000000000000003]
第三阶段的生产决策为:
[1, 1, 0, 0, 47501.36, 0, 0.0, 0.0]
2
最终的最大利润为95005977.46144022
第一阶段的生产决策为:
[0, 0, 0, 1, 217656.63999999996, 4214.0, 0.16000000000000003, 0.16000000000000003]
第二阶段的生产决策为:
[0, 0, 0, 1, 91720.50809599996, 1775.0, 0.16000000000000003, 0.16000000000000003]
第三阶段的生产决策为:
[1, 1, 0, 0, 49035.43999999999, 0, 0.0, 0.0]
3
最终的最大利润为97540654.5520002
第一阶段的生产决策为:
[0, 0, 0, 1, 208201.59999999992, 4355.0, 0.16000000000000003, 0.16000000000000003]
第二阶段的生产决策为:
[0, 0, 0, 1, 90671.79679999997, 1896.0, 0.16000000000000003, 0.16000000000000003]
第三阶段的生产决策为:
[1, 1, 0, 0, 50379.200000000004, 0, 0.0, 0.0]
4
最终的最大利润为99833130.96063991
第一阶段的生产决策为:
[0, 0, 0, 1, 198746.56, 4496.0, 0.16000000000000003, 0.16000000000000003]
第二阶段的生产决策为:
[0, 0, 0, 1, 89356.45337599999, 2021.0, 0.16000000000000003, 0.1600000

In [30]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:27
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:98
在0.8-1.2波动时,决策差异率为:0.784
